In [1]:
!pip install tqdm

In [2]:
from qiskit import *
from qiskit.visualization import plot_histogram
from numpy.random import randint
import numpy as np
from math import pi, acos

from tqdm import tqdm

In [3]:
n = 100; # number of qubits used in the BB84
np.random.seed(seed=123)
## Step 1
# Alice generates bits
alice_bits = randint(2, size=n)

print(alice_bits)
print("Alice's first bit = %i" % alice_bits[0])

[0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 1 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1
 0 0 1 0 1 0 1 1 1 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 0 1 0 0 1 1 1
 1 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 1 1 1 0 1 1 1 0 0 0]
Alice's first bit = 0


In [4]:
def encode_message(bits, bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        else: # Prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

In [5]:
def encode_message_bonus(bits, bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        elif bases[i] == 1: # Prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        else:
            if bits[i] == 0:
                qc.u3(pi/2, pi/2, pi, 0)
            else:
                qc.x(0)
                qc.u3(pi/2, pi/2, pi, 0)
            
        qc.barrier()
        message.append(qc)
    return message

In [6]:
## Step 2
# Create an array to tell us which qubits are encoded in which bases
alice_bases = randint(2, size=n)
message = encode_message(alice_bits, alice_bases)
print(alice_bases)
print("Alice's first chosen basis = %i" % alice_bases[0])

[1 0 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1 1 0 0 1 1 0 0
 0 1 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0 1 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 0 1
 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0]
Alice's first chosen basis = 1


In [7]:
def intercept_message(message, bases):
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
            message[q].h(0) # preparing the post-measurement state
        result = execute(message[q], backend, shots=1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [8]:
def intercept_message_v2(message, bases):
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for q in range(n):
        message[q].ry(pi*(3/4), 0)
        message[q].measure(0,0)
        message[q].ry(pi*(-3/4), 0) # preparing the post-measurement state
        result = execute(message[q], backend, shots=1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [9]:
def intercept_message_bonus(message, bases):
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
            message[q].h(0) # preparing the post-measurement state
        if bases[q] == 2:
            message[q].u3(pi/2, 0, pi/2, 0)
            message[q].measure(0, 0)
            message[q].u3(pi/2, pi/2, pi, 0)
        result = execute(message[q], backend, shots=1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [21]:
def intercept_message_bonus_v2(message, bases):
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for q in range(n):
        message[q].u3(-acos(1/(3**(1/2))), -pi, -pi/4, 0)
        message[q].measure(0, 0)
        message[q].u3(acos(1/(3**(1/2))), pi/4, pi, 0)
        result = execute(message[q], backend, shots=1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [11]:
## Interception!!
eve_bases = randint(2, size=n)
intercepted_message = intercept_message(message, eve_bases)
print(eve_bases)
print("Eve's first chosen basis = %i" % alice_bases[0])
print(intercepted_message)
print("Eve's first intercepted message = %i" % intercepted_message[0])

[1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 1 0
 1 0 0 0 1 1 1 0 1 0 0 1 1 1 0 1 1 0 0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 0 1
 1 0 0 0 1 1 0 0 1 0 0 0 1 0 1 1 1 0 1 1 1 0 0 0 0 1]
Eve's first chosen basis = 1
[0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1]
Eve's first intercepted message = 0


In [12]:
def measure_message(message, bases):
    backend = Aer.get_backend('aer_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(message[q], shots=1, memory=True)
        result = aer_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [13]:
def measure_message_bonus(message, bases):
    backend = Aer.get_backend('aer_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
        if bases[q] == 2:
            message[q].u3(pi/2, 0, pi/2, 0)
            message[q].measure(0, 0)
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(message[q], shots=1, memory=True)
        result = aer_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [14]:
## Step 3
# Decide which basis to measure in:
bob_bases = randint(2, size=n)
bob_results = measure_message(message, bob_bases)
print(bob_bases)
print("Bob's first chosen basis = %i" % bob_bases[0])

[0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 0 0 1 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 0 1 0
 0 1 0 1 1 0 0 0 0 1 1 1 1 1 0 1 0 1 1 1 0 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0
 0 1 1 0 1 0 1 0 1 1 0 0 0 1 0 1 1 0 0 1 0 0 1 1 0 0]
Bob's first chosen basis = 0


In [15]:
def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
    return good_bits

In [16]:
## Step 4
# Remove bits where Alice's chosen bases are not equal to that of Bob's
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
print(alice_key)
print(bob_key)

[0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0]


In [17]:
def sample_bits(bits, selection):
    sample = []
    for i in selection:
        # use np.mod to make sure the
        # bit we sample is always in 
        # the list range
        i = np.mod(i, len(bits))
        # pop(i) removes the element of the
        # list at index 'i'
        sample.append(bits.pop(i))
    return sample

In [26]:
# bonus : open 6 part ; close 6 part

interception_success = 0
try_inter = 3000
np.random.seed(seed=123)
_bit = 0

for i in tqdm(range(try_inter)):
    n = 30;
    # number of qubits used in the BB84
    ## Step 1
    # Alice generates bits
    alice_bits = randint(2, size=n)

    ## Step 2
    # Create an array to tell us which qubits are encoded in which bases
#     alice_bases = randint(2, size=n)
    
#     *** bonus ***
    alice_bases = randint(3, size=n)
#     *************
    
#     message = encode_message(alice_bits, alice_bases)
    
#     *** bonus ***
    message = encode_message_bonus(alice_bits, alice_bases)
#     *************

    ## Interception!!
#     eve_bases = randint(2, size=n)
    
#     *** bonus ***
    eve_bases = randint(3, size=n)
#     *************
    
#     intercepted_message = intercept_message(message, eve_bases)
#     intercepted_message = intercept_message_v2(message, eve_bases)
    
#     *** bonus ***
#     intercepted_message = intercept_message_bonus(message, eve_bases)
    intercepted_message = intercept_message_bonus_v2(message, eve_bases)
#     *************

    ## Step 3
    # Decide which basis to measure in:
#     bob_bases = randint(2, size=n)
#     bob_results = measure_message(message, bob_bases)
    
#     *** bonus ***
    bob_bases = randint(3, size=n)
#     *************
#     *** bonus ***
    bob_results = measure_message_bonus(message, bob_bases)
#     *************

    ## Step 4
    # Remove bits where Alice's chosen bases are not equal to that of Bob's
    alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
    bob_key = remove_garbage(alice_bases, bob_bases, bob_results)

    sample_size = int(len(alice_key)/2)
    _bit += sample_size
#     print(sample_size)
                    # Change this to something lower and see if
                    # Eve can intercept the message without Alice
                    # and Bob finding out
    bit_selection = randint(n, size=sample_size)
    bob_sample = sample_bits(bob_key, bit_selection)
    alice_sample = sample_bits(alice_key, bit_selection)

    if alice_sample == bob_sample:
        interception_success += 1
        
print("Successfully interception probability is : ", interception_success/try_inter)
print(_bit/try_inter)

  0%|                                                                                         | 0/3000 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp\ipykernel_37440\3283502570.py:18: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
  qc.u3(pi/2, pi/2, pi, 0)
C:\Users\User\AppData\Local\Temp\ipykernel_37440\3283502570.py:21: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
  

Successfully interception probability is :  0.17766666666666667
4.734333333333334


In [25]:
print(0.39**(1/2.35))

0.6698613789812733
